In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import ee, eemont
from forestry_carbon_arr.core import ForestryCarbonARR
from forestry_carbon_arr.utils.zarr_utils import save_dataset_efficient_zarr, load_dataset_zarr

import gcsfs
import os

fs = gcsfs.GCSFileSystem(project=os.getenv("GOOGLE_CLOUD_PROJECT"), token='/usr/src/app/user_id.json')


forestry = ForestryCarbonARR(config_path='./00_input/korindo.json')
forestry.initialize_gee()

✓ GEE Initialized successfully
  Credentials Path: /usr/src/app/user_id.json - loaded successfully


In [3]:
# aoi
from forestry_carbon_arr.core.utils import DataUtils
import geopandas as gpd
import geemap

data_utils = DataUtils(forestry.config, use_gee=True)
aoi_gpd, aoi_ee = data_utils.load_geodataframe_gee(forestry.config["AOI_path"])

aoi_gpd_utm = aoi_gpd.to_crs(epsg=32749)

print(f"✅ AOI loaded: {len(aoi_gpd_utm)} features")
print(f"   Area: {aoi_gpd_utm.geometry.area.sum()/10000:.2f} hectares")

/opt/venv/lib/python3.12/site-packages/geemap/conversion.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


✅ AOI loaded: 1 features
   Area: 144217.67 hectares


In [4]:
### load the exported gee image
asset_monthly_interpolated = 'projects/remote-sensing-476412/assets/korindo_smooth_monthly'

In [41]:
# !pip install lt-gee-py

In [6]:
import ee, eemont
from ltgee import LandTrendr

In [32]:
use_yearly_ee = True
if use_yearly_ee != True:
    monthly_agg = ee.ImageCollection(asset_monthly_interpolated)
    
    # Server-side list of unique years derived from system:time_start
    year_list = (
        monthly_agg
            .aggregate_array('system:time_start')         # ee.List of millis
            .map(lambda ts: ee.Date(ts).get('year'))      # convert to year
            .distinct()                                   # keep unique values
            .sort()                                       # optional
    )
    
    def annual_col_median(img_col, years):
        def per_year(year):
            start = ee.Date.fromYMD(year, 1, 1)
            end = start.advance(1, 'year')
            return (
                img_col
                .filterDate(start, end)
                .median()
                .set('year', year)
                .set('system:time_start', start.millis())
            )
        return ee.ImageCollection(years.map(per_year))
    
    ee_col_year_median = annual_col_median(monthly_agg, year_list)
else:
    monthly_agg = ee.ImageCollection(asset_monthly_interpolated)
    
    # Server-side list of unique years derived from system:time_start
    year_list = (
        monthly_agg
            .aggregate_array('system:time_start')         # ee.List of millis
            .map(lambda ts: ee.Date(ts).get('year'))      # convert to year
            .distinct()                                   # keep unique values
            .sort()                                       # optional
    )
    year_list = year_list.getInfo()
    print(year_list)

[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]


In [8]:
# ee_col_year_median.first().bandNames().getInfo()

In [9]:
# ee_col_year_median.first().propertyNames().getInfo()

In [10]:
# ee_col_year_median.first().get('system:time_start').getInfo()

In [11]:
# ### wmts

# ## skip this for gcp docker environment (visual is not needed yet into fastapi)

# from wmts_manager import WMTSManager

# wmts = WMTSManager(project_name=forestry.config['project_name'], aoi=aoi_ee.geometry())

# # Filter for August 2025
# aug_2025_img = ee_col_year_median.filterDate('2025-01-01', '2025-12-31').first()

# # Use a descriptive layer name (no getInfo needed)
# layer_name = 'annual_median_aug_2025'

# wmts.addLayer(
#     aug_2025_img, 
#     {
#         'bands': ['swir2', 'nir', 'red'],
#         'min': 0,
#         'max': 0.6,
#         'gamma': 1.5
#     }, 
#     layer_name
# )

# wmts.publish()

In [12]:
# ee_col_year_median

In [13]:
### look for the availability of the spectral indices in the eemont
# Dynamically get all available spectral indices from eemont-osi
# import sys
# sys.path.insert(0, '/usr/src/app/eemont-osi')
import ee_extra.Spectral.core as spec_core

# Get all available indices dynamically
indices_dict = spec_core.indices(online=False)
spectral_indices_awesome_list = sorted(list(indices_dict.keys()))

print(f"✅ Loaded {len(spectral_indices_awesome_list)} spectral indices from eemont-osi")
print(f"\nFirst 10 indices: {spectral_indices_awesome_list[:10]}")
print(f"\nLast 10 indices: {spectral_indices_awesome_list[-10:]}")
print(f"\n✅ List ready to use: spectral_indices_awesome_list")

# Function to get formula and metadata for any spectral index
def get_index_info(index_name):
    """
    Get formula and metadata for a spectral index.
    
    Parameters:
    -----------
    index_name : str
        Name of the spectral index (e.g., 'NDVI', 'EVI', 'SAVI')
    
    Returns:
    --------
    dict : Dictionary containing:
        - formula: Mathematical formula string
        - long_name: Full name of the index
        - bands: List of bands used (N=NIR, R=Red, G=Green, B=Blue, S1=SWIR1, S2=SWIR2, RE1-4=Red Edge)
        - application_domain: Category (vegetation, water, burn, etc.)
        - platforms: Supported satellite platforms
        - reference: Reference URL or DOI
    """
    index_name_upper = index_name.upper()
    
    if index_name_upper not in indices_dict:
        available = [idx for idx in spectral_indices_awesome_list if index_name.upper() in idx.upper()]
        raise ValueError(
            f"Index '{index_name}' not found. "
            f"Did you mean: {available[:5] if available else 'None'}?"
        )
    
    info = indices_dict[index_name_upper].copy()
    return info

def formula(index_name):
    """
    Get the formula for a spectral index.
    
    Parameters:
    -----------
    index_name : str
        Name of the spectral index (e.g., 'NDVI', 'EVI', 'SAVI')
    
    Returns:
    --------
    str : Mathematical formula string using band abbreviations
         Band abbreviations: N (NIR), R (Red), G (Green), B (Blue), 
         S1 (SWIR1), S2 (SWIR2), RE1-4 (Red Edge 1-4)
    
    Examples:
    --------
    >>> formula('NDVI')
    '(N - R)/(N + R)'
    
    >>> formula('EVI')
    'G * ((N - R) / (N + C1 * R - C2 * B + L))'
    """
    info = get_index_info(index_name)
    return info['formula']

# Function to convert formula band abbreviations to OSI band names
def formula_to_osi_bands(formula_str):
    """
    Convert eemont-osi formula band abbreviations to OSI band names.
    
    Mapping:
    - N (NIR) -> nir
    - R (Red) -> red
    - G (Green) -> green
    - B (Blue) -> blue
    - S1 (SWIR1) -> swir1
    - S2 (SWIR2) -> swir2
    - RE1 (Red Edge 1) -> redE1
    - RE2 (Red Edge 2) -> redE2
    - RE3 (Red Edge 3) -> redE3
    - RE4 (Red Edge 4) -> redE4
    - Variables (g, C1, C2, L, etc.) remain as-is
    
    Parameters:
    -----------
    formula_str : str
        Formula string from eemont-osi (e.g., "(N - R)/(N + R)")
    
    Returns:
    --------
    str : Formula with OSI band names (e.g., "(nir - red)/(nir + red)")
    """
    import re
    
    # Mapping from eemont-osi abbreviations to OSI band names
    band_mapping = {
        'N': 'nir',      # Near Infrared
        'R': 'red',      # Red
        'G': 'green',    # Green
        'B': 'blue',     # Blue
        'S1': 'swir1',   # Shortwave Infrared 1
        'S2': 'swir2',   # Shortwave Infrared 2
        'RE1': 'redE1',  # Red Edge 1
        'RE2': 'redE2',  # Red Edge 2
        'RE3': 'redE3',  # Red Edge 3
        'RE4': 'redE4',  # Red Edge 4
    }
    
    # Sort by length (longest first) to avoid partial matches (e.g., RE1 before R)
    sorted_bands = sorted(band_mapping.keys(), key=len, reverse=True)
    
    result = formula_str
    
    # Replace band abbreviations with OSI names
    # Use word boundaries to avoid replacing partial matches in variables
    for abbrev in sorted_bands:
        osi_name = band_mapping[abbrev]
        # Use regex to match whole words only (not part of other words)
        # Pattern: \b matches word boundary, but we need to handle cases like "RE1" in "RE1*RE2"
        pattern = r'\b' + re.escape(abbrev) + r'\b'
        result = re.sub(pattern, osi_name, result)
    
    return result

def formula_osi(index_name):
    """
    Get the formula for a spectral index with OSI band names.
    
    Parameters:
    -----------
    index_name : str
        Name of the spectral index (e.g., 'NDVI', 'EVI', 'NBR')
    
    Returns:
    --------
    str : Mathematical formula string using OSI band names
         (nir, red, green, blue, swir1, swir2, redE1-4)
    
    Examples:
    --------
    >>> formula_osi('NDVI')
    '(nir - red)/(nir + red)'
    
    >>> formula_osi('NBR')
    '(nir - swir2)/(nir + swir2)'
    """
    formula_orig = formula(index_name)
    return formula_to_osi_bands(formula_orig)

# Example usage
print("\n" + "="*60)
print("Example: Getting formula for NDVI and NBR")
print("="*60)
try:
    # NDVI example
    ndvi_formula = formula('NDVI')
    ndvi_formula_osi = formula_osi('NDVI')
    ndvi_info = get_index_info('NDVI')
    print(f"\nNDVI:")
    print(f"  Original Formula: {ndvi_formula}")
    print(f"  OSI Band Names: {ndvi_formula_osi}")
    print(f"  Long Name: {ndvi_info['long_name']}")
    print(f"  Bands: {ndvi_info['bands']}")
    
    # NBR example (shows S2 -> swir2)
    nbr_formula = formula('NBR')
    nbr_formula_osi = formula_osi('NBR')
    nbr_info = get_index_info('NBR')
    print(f"\nNBR (Normalized Burn Ratio):")
    print(f"  Original Formula: {nbr_formula}")
    print(f"  OSI Band Names: {nbr_formula_osi}")
    print(f"  Long Name: {nbr_info['long_name']}")
    print(f"  Bands: {nbr_info['bands']} (S2 = SWIR2)")
    print(f"  Domain: {nbr_info['application_domain']}")
    
except Exception as e:
    print(f"Error: {e}")



✅ Loaded 253 spectral indices from eemont-osi

First 10 indices: ['AFRI1600', 'AFRI2100', 'ANDWI', 'ARI', 'ARI2', 'ARVI', 'ATSAVI', 'AVI', 'AWEInsh', 'AWEIsh']

Last 10 indices: ['kNDVI', 'kRVI', 'kVARI', 'mND705', 'mSR705', 'sNIRvLSWI', 'sNIRvNDPI', 'sNIRvNDVILSWIP', 'sNIRvNDVILSWIS', 'sNIRvSWIR']

✅ List ready to use: spectral_indices_awesome_list

Example: Getting formula for NDVI and NBR

NDVI:
  Original Formula: (N - R)/(N + R)
  OSI Band Names: (nir - red)/(nir + red)
  Long Name: Normalized Difference Vegetation Index
  Bands: ['N', 'R']

NBR (Normalized Burn Ratio):
  Original Formula: (N - S2) / (N + S2)
  OSI Band Names: (nir - swir2) / (nir + swir2)
  Long Name: Normalized Burn Ratio
  Bands: ['N', 'S2'] (S2 = SWIR2)
  Domain: burn


In [14]:
# spectral_indices_awesome_list

In [18]:
if use_yearly_ee != True:
    ee_col_year_median_with_indices = ee_col_year_median.spectralIndices(
        index=['EVI', 'GNDVI', 'SAVI','NDVI','NBR','VARI','NDWI','MTVI2'],
        satellite_type='Sentinel',  # OSI-style satellite type
        G=2.5,  # EVI parameters
        C1=6.0,
        C2=7.5,
        L=1.0,  # SAVI parameter
        drop=False  # Keep original bands
    )

In [16]:
# ee_col_year_median_with_indices.first().bandNames().getInfo()

In [19]:
if use_yearly_ee != True:
    # Server-side list of unique years derived from system:time_start
    year_list = (
        ee_col_year_median_with_indices
            .aggregate_array('system:time_start')         # ee.List of millis
            .map(lambda ts: ee.Date(ts).get('year'))      # convert to year
            .distinct()                                   # keep unique values
            .sort()                                       # optional
    )
    year_list_c = year_list.getInfo()
    year_list_c

In [20]:
# year_list_c[0]

In [21]:
forestry.config

{'project': {'name': 'forestry_carbon_project',
  'region': 'asia',
  'description': 'Forestry Carbon ARR Analysis Project',
  'version': '1.0.0'},
 'gee': {'project_id': None,
  'service_account': None,
  'initialize': True,
  'max_pixels': 10000000000000.0,
  'scale': 30,
  'crs': 'EPSG:4326'},
 'satellite': {'provider': 'Custom',
  'date_range': ['2025-8-1', '2025-8-31'],
  'cloud_cover_threshold': 80,
  'bands': {'Sentinel': ['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],
   'Planet': ['red', 'green', 'blue', 'nir'],
   'Landsat': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']},
  'composite_method': 'median',
  'mask_clouds': True},
 'ml': {'algorithm': 'gbm',
  'training_samples': 1000,
  'validation_split': 0.2,
  'cross_validation': True,
  'n_folds': 5,
  'random_state': 42,
  'hyperparameter_tuning': True},
 'fcd': {'method': 'pca',
  'thresholds': {'high_forest': 65,
   'yrf_forest': 45,
   'shrub_grass': 45,
   'open_land': 30},
  'apply_smoothing': True,
  'smoothing_kernel': 3},
 'classifi

In [22]:
if use_yearly_ee != True:
    ## do the FCD every year
    from gee_lib.osi.fcd import FCDCalc
    
    list_fcd_year = []
    
    forestry.config['AOI'] = aoi_ee
    
    year_0 = year_list_c[0]
    fcd = FCDCalc(forestry.config, image_mosaick=ee_col_year_median_with_indices.filterDate(f'{year_0}-1-1',f'{year_0}-12-31').first())
    for i, year in enumerate(year_list_c):
        print(f'processing year {year}')
        if i ==0:
            fcd_2_1 = fcd.fcd_calc()['FCD2_1']
        else:
            fcd.image_mosaick = ee_col_year_median_with_indices.filterDate(f'{year}-1-1',f'{year}-12-31').first()
            fcd_2_1 = fcd.fcd_calc()['FCD2_1']
        fcd_2_1 = fcd_2_1.set('year', year).set('system:time_start', ee.Date.fromYMD(year, 1, 1).millis())
        
        list_fcd_year.append(fcd_2_1)

In [23]:
# list_fcd_year[0].bandNames().getInfo()

In [24]:
# list_fcd_year[0].propertyNames().getInfo()

In [25]:
# list_fcd_year[0].get('year')

In [26]:
from wfs_manager import WFSManager

AOI = ee.FeatureCollection('projects/remote-sensing-476412/assets/korindo_with_buffer')

wfs = WFSManager(fastapi_url="http://fastapi:8000", wfs_base_url="http://localhost:8001")
wfs.addLayer(AOI, "AOI Boundary")
wfs.publish()

{'status': 'success',
 'total_layers': 1,
 'successful_layers': 1,
 'failed_layers': 0,
 'layers': {'aoi_boundary': {'status': 'success',
   'layer_name': 'aoi_boundary',
   'fc_url': 'http://fastapi:8000/fc/aoi_boundary',
   'wfs_url': 'http://localhost:8001/wfs?service=WFS&version=1.1.0&request=GetFeature&typename=aoi_boundary',
   'feature_count': 1,
   'response': {'message': "FeatureCollection 'aoi_boundary' created successfully",
    'name': 'aoi_boundary',
    'type': 'FeatureCollection',
    'count': 1}}},
 'service_urls': {'wfs_capabilities': 'http://localhost:8001/wfs?service=WFS&version=1.1.0&request=GetCapabilities',
  'wfs_base': 'http://localhost:8001/wfs'}}

In [27]:
# fcd_2015 = list_fcd_year[0]

# # Use a descriptive layer name (no getInfo needed)
# layer_name = 'fcd_2015'

# wmts.addLayer(
#     fcd_2015, 
#     {'min':0 ,'max':80, 'palette':['ff4c16', 'ffd96c', '39a71d']}, 
#     layer_name
# )

# wmts.publish()


In [28]:
import ee

# Example: Export single image to GCS
def export_image_to_gcs(image, gcs_bucket, gcs_path, scale=10, crs='EPSG:4326', region=None, max_pixels=1e13):
    """
    Export Earth Engine Image to Google Cloud Storage
    
    Parameters:
    -----------
    image : ee.Image
        Earth Engine Image to export
    gcs_bucket : str
        GCS bucket name (e.g., 'my-bucket' or 'gs://my-bucket')
    gcs_path : str
        Path within bucket (e.g., 'exports/fcd_2020.tif')
    scale : float
        Pixel scale in meters (default: 30)
    crs : str
        Coordinate reference system (default: 'EPSG:4326')
    region : ee.Geometry, optional
        Region to export (default: None, uses image bounds)
    max_pixels : int
        Maximum pixels to export (default: 1e9)
    
    Returns:
    --------
    ee.batch.Task : Export task
    """
    # Clean bucket name (remove gs:// if present)
    if gcs_bucket.startswith('gs://'):
        gcs_bucket = gcs_bucket.replace('gs://', '').split('/')[0]
    
    # Full GCS path
    gcs_uri = f"gs://{gcs_bucket}/{gcs_path}"
    
    # Export parameters
    export_params = {
        'image': image,
        'description': gcs_path.split('/')[-1].replace('.tif', ''),  # Task name
        'bucket': gcs_bucket,
        'fileNamePrefix': gcs_path.replace('.tif', ''),  # Path without extension
        'scale': scale,
        'crs': crs,
        'maxPixels': max_pixels,
        'fileFormat': 'GeoTIFF',
        'formatOptions': {
            'cloudOptimized': True  # COG format
        }
    }
    
    # Add region if provided
    if region is not None:
        export_params['region'] = region
    
    # Create export task
    task = ee.batch.Export.image.toCloudStorage(**export_params)
    
    # Start the task
    task.start()
    
    print(f"✅ Export task started: {gcs_uri}")
    print(f"   Task ID: {task.id}")
    
    return task

In [29]:
if use_yearly_ee != True:
    ## collection will be export to the gcs as geotif
    for i, img in enumerate(list_fcd_year):
        year = img.get('year').getInfo()
        
        task = export_image_to_gcs(
            image=img,
            gcs_bucket='remote_sensing_saas',
            gcs_path=f'01-korindo/yearly_mosaic_gee/fcd_{year}.tif',
            scale=10,
            crs='EPSG:32749',  # UTM zone for your AOI
            region=aoi_ee.geometry()  # Optional: clip to AOI
        )

In [33]:
### wait until all the fcd is processed (exported)!
# ============================================
# STEP 2: Load directly from GCS when needed
# ============================================
# Later, when you need to use the images:
def load_yearly_images_from_gcs(years, gcs_bucket='remote_sensing_saas', base_path='01-korindo/yearly_mosaic_gee'):
    """
    Load yearly images directly from GCS without ingesting to GEE.
    
    Parameters:
    -----------
    years : list[int]
        List of years to load (e.g., [2020, 2021, 2022])
    gcs_bucket : str
        GCS bucket name
    base_path : str
        Base path in bucket
    
    Returns:
    --------
    ee.ImageCollection : Collection of images loaded from GCS
    """
    images = []
    for year in years:
        gcs_path = f'gs://{gcs_bucket}/{base_path}/fcd_{year}.tif'
        img = ee.Image.loadGeoTIFF(gcs_path)
        img = img.set('year', year)
        images.append(img)
    
    return ee.ImageCollection(images)
fcd_col = load_yearly_images_from_gcs(year_list)
fcd_col.first().bandNames().getInfo()

['FCD']

In [35]:
year_list[-1]

2025

In [43]:
# Prepare collection: must have 'system:time_start' and the FCD band
# Select FCD band and ensure time property is set
fcd_collection_lt = fcd_col.select('FCD').sort('system:time_start')

# LandTrendr parameters (for FCD 0-100 range)
lt_params = {
    'maxSegments': 6,
    'spikeThreshold': 0.85,
    'vertexCountOvershoot': 3,
    'preventOneYearRecovery': True,
    'recoveryThreshold': 15,  # 15 FCD points (not 0.15!)
    'pvalThreshold': 0.05,
    'bestModelProportion': 0.75
}

# Run LandTrendr using GEE's native algorithm
lt_result = ee.Algorithms.TemporalSegmentation.LandTrendr(
    timeSeries=fcd_collection_lt,
    **lt_params
)

print("✅ LandTrendr completed!")
print(f"Result bands: {lt_result.bandNames().getInfo()}")

✅ LandTrendr completed!
Result bands: ['LandTrendr', 'rmse']
